# Telegram Uploader - Colab Version

This notebook allows you to upload files to a Telegram channel from Google Colab using your API credentials.

In [ ]:
# Install dependencies
!pip install telethon python-dotenv nest_asyncio

In [ ]:
import os
import asyncio
import nest_asyncio
from telethon import TelegramClient, errors
from google.colab import drive

# Apply nest_asyncio to allow nested event loops in Colab
nest_asyncio.apply()

# Mount Google Drive to access files easily
drive.mount('/content/drive')

In [ ]:
# Configuration (Prompts for inputs)
try:
    API_ID = int(input("Enter API_ID: "))
    API_HASH = input("Enter API_HASH: ")
    PHONE_NUMBER = input("Enter PHONE_NUMBER (e.g., +1234567890): ")
except ValueError:
    print("Invalid input. API_ID must be an integer.")

SESSION_NAME = 'colab_telegram_session'

In [ ]:
async def upload_file_colab(channel_username, file_path, caption=None):
    """
    Uploads a file to a Telegram channel in Colab.
    """
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return

    print(f"Connecting to Telegram...")
    # Initialize client
    client = TelegramClient(SESSION_NAME, API_ID, API_HASH)
    
    # Start the client (handles interactive login in output)
    await client.start(phone=PHONE_NUMBER)

    async with client:
        # Resolve the entity first
        try:
            if str(channel_username).lstrip('-').isdigit():
                 entity_input = int(channel_username)
            else:
                 entity_input = channel_username
            
            entity = await client.get_entity(entity_input)
            print(f"Resolved entity: {entity.title} (ID: {entity.id})")
        except ValueError:
            print(f"Error: Could not find entity '{channel_username}'. Ensure you are subscribed.")
            return
        except Exception as e:
            print(f"Error resolving entity: {e}")
            return

        print(f"Uploading {file_path} to {entity.title}...")
        
        def progress_callback(current, total):
            print(f"Uploaded: {current / total * 100:.1f}%", end='\r')

        try:
            await client.send_file(
                entity,
                file_path,
                caption=caption,
                progress_callback=progress_callback
            )
            print(f"\nUpload complete!")
        
        except errors.FloodWaitError as e:
            print(f"\nRate limit exceeded. Waiting for {e.seconds} seconds...")
            await asyncio.sleep(e.seconds)
            print("Retrying upload...")
            await client.send_file(
                entity,
                file_path,
                caption=caption,
                progress_callback=progress_callback
            )
            print(f"\nUpload complete!")

        except errors.RPCError as e:
             print(f"\nTelegram RPC Error: {e} (Code: {e.code})")
             if "Too many requests" in str(e):
                 print("Hint: Server-side limit. Try again later.")

        except Exception as e:
            print(f"\nError uploading file: {type(e).__name__}: {e}")

In [ ]:
# Run the Upload
CHANNEL_ID_OR_USERNAME = input("Enter Channel ID/Username: ")
FILE_PATH = input("Enter File Path (e.g., /content/drive/MyDrive/video.mp4): ")
CAPTION = input("Enter Caption (optional): ") or None

if CHANNEL_ID_OR_USERNAME and FILE_PATH:
    await upload_file_colab(CHANNEL_ID_OR_USERNAME, FILE_PATH, CAPTION)
else:
    print("Please provide channel ID and file path.")